In [30]:
import os
print(os.getcwd())


C:\Users\Saniya\Desktop\Customer churn ML end to end\notebooks


In [31]:
import os
import pandas as pd
import numpy as np


# 2) Load dataset
DATA_PATH = "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(DATA_PATH)

# 3) Basic audit
print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())
print("\nDtypes:\n", df.dtypes)

print("\nInfo():")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False).head(10))

# 4) Target check
print("\nTarget distribution (Churn):")
print(df['Churn'].value_counts(dropna=False))
print("Target ratio (Churn=Yes):", (df['Churn'] == 'Yes').mean())

# 5) Suspect column check: TotalCharges should be numeric but is often object
print("\nTotalCharges dtype:", df['TotalCharges'].dtype)
if df['TotalCharges'].dtype == 'O':
    blanks = (df['TotalCharges'].str.strip() == '').sum()
    print("Blank strings in TotalCharges:", blanks)

# 6) Quick peek
df.head(3)


Shape: (7043, 21)

Columns:
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Dtypes:
 customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Info():
<class 'pandas.core.frame.

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [32]:
#DONT RUN THIS THE ORIGNAL 2.2 WAS DELETED WHERE TOTALCHARGES WERE CONVERT TO NUMBER BAD COLUMNS WERE DROPPED AND CUSTOMERID WAS DROPPED AND CHURN WAS CONVERTED TO 0 AND 1
# Step 2.2 – Fix Columns & Missing Values

# 1) Strip whitespace from all string columns (defensive cleanup)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# 2) Convert TotalCharges to numeric (invalid values -> NaN)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 3) Drop rows with missing TotalCharges (expected: 11 rows)
before_rows = df.shape[0]
df = df.dropna(subset=['TotalCharges'])
after_rows = df.shape[0]
print(f"Dropped {before_rows - after_rows} rows with missing TotalCharges")

# 4) Drop customerID (identifier, not useful for ML)
df = df.drop(columns=['customerID'])

# 5) Encode target variable Churn (Yes=1, No=0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 6) Convert SeniorCitizen to categorical (even though stored as 0/1)
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

# 7) Check results
print("Shape after cleaning:", df.shape)
print("Dtypes after cleaning:\n", df.dtypes)
print("\nChurn distribution:\n", df['Churn'].value_counts(normalize=True))

df.head(3)


Dropped 11 rows with missing TotalCharges
Shape after cleaning: (7032, 20)
Dtypes after cleaning:
 gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

Churn distribution:
 0    0.734215
1    0.265785
Name: Churn, dtype: float64


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1


In [34]:
# ✅ Verification checks after Step 2.2 cleaning

print("Shape:", df.shape)   # should be (7032, 20)

# 1) Check for customerID (should be dropped)
if "customerID" in df.columns:
    print("Dropping customerID...")
    df = df.drop(columns=["customerID"])
else:
    print("customerID already dropped ✅")

print("\nColumns:", df.columns.tolist())


# 2) Check TotalCharges dtype (should be float)
print("\nTotalCharges dtype:", df['TotalCharges'].dtype)
assert df['TotalCharges'].dtype in ['float64', 'int64'], "TotalCharges not numeric!"

# 3) Check for missing values (should be 0 in TotalCharges now)
print("\nMissing values per column:\n", df.isna().sum())

# 4) Check Churn encoding (should be 0/1 int)
print("\nChurn unique values:", df['Churn'].unique())
assert set(df['Churn'].unique()) <= {0,1}, "Churn not encoded as 0/1!"

# 5) Check SeniorCitizen dtype (should be object now)
print("\nSeniorCitizen dtype:", df['SeniorCitizen'].dtype)

# 6) Quick churn distribution
print("\nChurn distribution:\n", df['Churn'].value_counts(normalize=True))


Shape: (7032, 20)
customerID already dropped ✅

Columns: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

TotalCharges dtype: float64

Missing values per column:
 gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Churn unique values: [0 1]

SeniorCitizen dtype: object

Churn distribution:
 0    0.734215
1    0.265785
Name: Churn, dt

In [35]:
# Map SeniorCitizen values: 0 -> No, 1 -> Yes
df['SeniorCitizen'] = df['SeniorCitizen'].map({'0': 'No', '1': 'Yes', 0: 'No', 1: 'Yes'}).astype('object')

# Verify
print(df['SeniorCitizen'].value_counts())
print("dtype:", df['SeniorCitizen'].dtype)


No     5890
Yes    1142
Name: SeniorCitizen, dtype: int64
dtype: object
